In [23]:
%pip install -r requirements.txt
import os
from pdf_parser import *
from huggingface_hub import hf_hub_download
from PIL import Image 
import requests 
from transformers import AutoModelForCausalLM 
from transformers import AutoProcessor 


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [24]:
Images_list,Text_list = get_Text_and_Images(file_path="files/O-RAN.WG3.TS.E2AP-R004-v07.00 (1).pdf")
cleaned_images = clean_images(Images_list)
cleaned_text = clean_text(Text_list)
captioned_photos = add_captions(cleaned_images,cleaned_text)


In [25]:
# for img in captioned_photos:
#     display(Image(data=img['img']))
#     print(img['caption'],f"on page {img['page']}")

In [26]:
model_id = "microsoft/Phi-3.5-vision-instruct" 

In [38]:
model = AutoModelForCausalLM.from_pretrained(
  model_id, 
  device_map=None,
  trust_remote_code=True, 
  torch_dtype="auto", 
  _attn_implementation="eager"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  5.71it/s]


In [39]:
processor = AutoProcessor.from_pretrained(model_id, 
  trust_remote_code=True, 
  num_crops=4,
) 

/Users/atharva/miniconda3/envs/oranbot/lib/python3.11/site-packages/transformers/models/auto/image_processing_auto.py:513: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


In [40]:

images = []
placeholder = ""

In [41]:
for i in range(1,10):
    url = f"https://image.slidesharecdn.com/azureintroduction-191206101932/75/Introduction-to-Microsoft-Azure-Cloud-{i}-2048.jpg" 
    images.append(Image.open(requests.get(url, stream=True).raw))
    placeholder += f"<|image_{i}|>\n"


In [42]:
messages = [
    {"role": "user", "content": placeholder+"Summarize the deck of slides."},
]

In [43]:
prompt = processor.tokenizer.apply_chat_template(
  messages, 
  tokenize=False, 
  add_generation_prompt=True
)


In [44]:
inputs = processor(prompt, images, return_tensors="pt")

In [45]:
generation_args = { 
    "max_new_tokens": 1000, 
    "temperature": 0.0, 
    "do_sample": False, 
} 


In [46]:
generate_ids = model.generate(**inputs, 
  eos_token_id=processor.tokenizer.eos_token_id, 
  **generation_args
)

/Users/atharva/miniconda3/envs/oranbot/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, 
  skip_special_tokens=True, 
  clean_up_tokenization_spaces=False)[0] 


In [ ]:

print(response)